## Fast Charge 

ce notebook montre la dégradation du SoH à cause de la charge rapide.

Conclusion:
- on a 7988 vin parmis eux:
    - 1408 ont fait au moins une charge rapide.
    - 165 l'utilisent plus que la moitié du temps. 
    - 7644 l'utilisent moins d'un quart du temps.

On voit une dégradation du SoH 2x plus élevé pour les vin qui utilisent beaucoup la fast charge contrairement à ceux qui ne l'utilisent pas ou très peu.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import pandas as pd
from core.sql_utils import *

## Function

In [ ]:
def plot_log(df, column):
    def log_function(x, a):
        return 1 + a * np.log1p(x/1000) 
    fig = go.Figure()
    # create color
    model_colors = {value: px.colors.qualitative.Plotly[i] for i, value in enumerate(df[column].unique())}

    for value in df[column].unique():
        df_model_temp = df[df[column]==value].dropna(subset='soh').sort_values('odometer').copy()
        # fir log function
        popt, _ = curve_fit(log_function, df_model_temp['odometer'], df_model_temp['soh'])
        x_vals = np.linspace(0.1,  df_model_temp.odometer.max(), 500)
        y_vals = log_function(x_vals, *popt)

         # Couleur unique pour le modèle
        color = model_colors[value] 

        # Génération des valeurs ajustées
        fig.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{value} trend', line=dict(color=color)))
        
    return fig

## data

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b
            on b.id = vm.battery_id
            WHERE vm.model_name like '%model%';"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
level_charge = dbeaver_df.groupby('vin').agg(
    total_level_1=('level_1', 'sum'),
    total_level_2=('level_2', 'sum'),
    total_level_3=('level_3', 'sum'),
    soh = ('soh', 'last'),
    odometer=('odometer', 'last')
).eval('ratio_level_3=total_level_3/(total_level_3 +  total_level_2+  total_level_1)')

In [ ]:
level_charge["ratio_level_3_cat"] = level_charge['ratio_level_3'].apply(lambda x: "Using fast charging more than half the time" if x >=.5 else 
                                                                        "Using fast charging more than a quarter of the time" if .5> x >.25 
                                                                        else "Low use of fast charging")

## Result 

In [ ]:
level_charge[level_charge['ratio_level_3'] > 0].shape

In [ ]:
level_charge["ratio_level_3_cat"].value_counts()

In [ ]:
fig_fast_charge = plot_log(level_charge, "ratio_level_3_cat")
fig_fast_charge.update_layout(title='Battery degradation depending on the use of fast charging')